In [ ]:
import os
import shutil

def gather_and_rename_images(source_root, target_folder):
    # Create target subfolders if they don't exist
    images_folder = os.path.join(target_folder, 'images')
    masks_folder = os.path.join(target_folder, 'masks')
    masks_folder2 = os.path.join(target_folder, 'masks2')
    
    os.makedirs(images_folder, exist_ok=True)
    os.makedirs(masks_folder, exist_ok=True)
    os.makedirs(masks_folder2, exist_ok=True)  # FIXED: was missing

    for root, _, files in os.walk(source_root):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.JPG')):
                full_path = os.path.join(root, file)

                # Split path to extract parts
                parts = full_path.split(os.sep)

                try:
                    # Extract date and location from folder structure
                    # Expected: .../Location/Date/filename.jpg
                    location = parts[-3]
                    date = parts[-2]
                    base_filename, ext = os.path.splitext(file)

                    # Check if it's a mask or mask2
                    is_mask2 = 'mask2' in base_filename.lower()
                    is_mask = 'mask' in base_filename.lower() and not is_mask2

                    # Create new filename
                    new_filename = f"{base_filename}_{location}_{date}{ext}"

                    # Choose destination folder
                    if is_mask:
                        dest_path = os.path.join(masks_folder, new_filename)
                    elif is_mask2:
                        dest_path = os.path.join(masks_folder2, new_filename)
                    else:
                        dest_path = os.path.join(images_folder, new_filename)

                    # Copy the file
                    shutil.copy2(full_path, dest_path)
                    print(f"Copied to: {dest_path}")

                except IndexError:
                    print(f"Skipped (invalid path): {full_path}")

# Example usage
source = 'original_data_2025'
target = 'Data_2025'

gather_and_rename_images(source, target)


Copied to: Data/images/V1_2_par_30_La vicogne_0723.jpg
Copied to: Data/masks2/V3_1_perp_30_mask2_La vicogne_0723.JPG
Copied to: Data/images/V1_2_perp_30_La vicogne_0723.jpg
Copied to: Data/masks2/V3_2_par_30_mask2_La vicogne_0723.JPG
Copied to: Data/images/V3_0_par_57_La vicogne_0723.jpg
Copied to: Data/images/V1_2_perp_57_La vicogne_0723.jpg
Copied to: Data/images/V2_2_par_30_La vicogne_0723.jpg
Copied to: Data/masks2/V3_0_par_45_mask2_La vicogne_0723.JPG
Copied to: Data/masks/V3_0_par_57_mask_La vicogne_0723.JPG
Copied to: Data/images/V2_2_par_45_La vicogne_0723.jpg
Copied to: Data/masks2/V2_0_par_57_mask2_La vicogne_0723.JPG
Copied to: Data/images/V2_0_par_57_La vicogne_0723.jpg
Copied to: Data/images/V1_0_par_0_La vicogne_0723.jpg
Copied to: Data/masks/V1_2_par_57_mask_La vicogne_0723.JPG
Copied to: Data/masks/V3_0_perp_0_mask_La vicogne_0723.JPG
Copied to: Data/images/V3_2_perp_57_La vicogne_0723.jpg
Copied to: Data/masks/V2_2_par_45_mask_La vicogne_0723.JPG
Copied to: Data/masks/

In [24]:
import os
import pandas as pd

def merge_label_csvs(source_root, output_filename='label.csv'):
    all_data = []

    # Walk through all subdirectories
    for root, _, files in os.walk(source_root):
        for file in files:
            if file.lower().endswith('.csv'):
                full_path = os.path.join(root, file)
                try:
                    df = pd.read_csv(full_path)

                    if df.empty:
                        print(f"⚠️ Skipped empty file: {full_path}")
                        continue

                    all_data.append(df)
                    print(f"✅ Loaded: {full_path} ({len(df)} rows)")

                except Exception as e:
                    print(f"❌ Failed to load {full_path}: {e}")

    # Final check before merging
    if not all_data:
        print("❌ No valid CSV files found.")
        return

    # Merge and save
    merged_df = pd.concat(all_data, ignore_index=True)
    output_path = os.path.join(source_root, output_filename)
    merged_df.to_csv(output_path, index=False)
    print(f"\n✅ All CSVs merged into: {output_path} ({len(merged_df)} total rows)")

# Example usage:
merge_label_csvs("original_data")


✅ Loaded: original_data/La vicogne/0716.csv (72 rows)
✅ Loaded: original_data/La vicogne/0628.csv (24 rows)
✅ Loaded: original_data/La vicogne/0708.csv (72 rows)
✅ Loaded: original_data/La vicogne/0723.csv (72 rows)
✅ Loaded: original_data/La vicogne/0813.csv (72 rows)
✅ Loaded: original_data/Lataule/0613.csv (72 rows)
✅ Loaded: original_data/Lataule/0627.csv (120 rows)
✅ Loaded: original_data/Lataule/0724.csv (136 rows)
✅ Loaded: original_data/Lataule/0614.csv (72 rows)
✅ Loaded: original_data/Lataule/0709.csv (136 rows)
✅ Loaded: original_data/Lataule/0813.csv (72 rows)
✅ Loaded: original_data/Lataule/0711.csv (72 rows)
✅ Loaded: original_data/Lataule/0717.csv (96 rows)

✅ All CSVs merged into: original_data/label.csv (1088 total rows)


In [25]:
import pandas as pd

def fix_backslashes_in_paths(csv_path, output_csv=None):
    # Load CSV
    df = pd.read_csv(csv_path)

    # Replace backslashes with slashes in img_path column
    df['img_path'] = df['img_path'].astype(str).str.replace('\\', '/', regex=False)

    # Save to new or same file
    if output_csv is None:
        output_csv = csv_path  # Overwrite the original

    df.to_csv(output_csv, index=False)
    print(f"✅ Paths fixed and saved to: {output_csv}")

# Example usage
fix_backslashes_in_paths("label.csv")


✅ Paths fixed and saved to: label.csv


In [26]:
import pandas as pd
import os

def clean_image_paths(csv_path, output_csv=None):
    df = pd.read_csv(csv_path)

    def transform_path(row):
        original_path = row['img_path'].replace('\\', '/')  # Force forward slashes

        # Split the normalized path
        parts = original_path.split('/')

        try:
            # Extract filename and name part
            filename = os.path.basename(original_path)
            name_part = os.path.splitext(filename)[0]

            # Extract location and date
            date = parts[-2]             # e.g., '0628'
            location = parts[-4]         # e.g., 'La vicogne'

            # Build new filename
            new_filename = f"{name_part}_{location}_{date}.jpg"

            # New full path
            new_path = f"C:/stage_part3/Data/{new_filename}"
            return new_path

        except IndexError:
            print(f"⚠️ Could not parse path: {original_path}")
            return original_path

    # Apply transformation
    df['img_path'] = df.apply(transform_path, axis=1)

    # Save the corrected CSV
    if output_csv is None:
        output_csv = csv_path  # Overwrite original

    df.to_csv(output_csv, index=False)
    print(f"✅ Cleaned paths saved to: {output_csv}")

# Example usage
clean_image_paths("label.csv")


✅ Cleaned paths saved to: label.csv


In [30]:
import os
import shutil

def copy_images_with_matching_masks(images_dir, masks_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    image_filenames = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]
    mask_filenames = set(os.listdir(masks_dir))  # for fast lookup

    copied, skipped = 0, 0

    for image_name in image_filenames:
        name_base, ext = os.path.splitext(image_name)

        # Insert '_mask' before the last two parts separated by '_'
        parts = name_base.split('_')
        if len(parts) < 3:
            print(f"⚠️ Skipping malformed filename: {image_name}")
            skipped += 1
            continue

        # Insert "_mask" before the last two segments (location and date)
        mask_base = '_'.join(parts[:-2] + ['mask'] + parts[-2:])
        mask_name = f"{mask_base}{ext}"

        if mask_name in mask_filenames:
            shutil.copy2(os.path.join(images_dir, image_name),
                         os.path.join(output_dir, image_name))
            copied += 1
        else:
            skipped += 1

    print(f"✅ {copied} images copied.")
    print(f"⏭️ {skipped} images skipped (no matching mask).")

# Example usage
copy_images_with_matching_masks(
    images_dir='Data/images',
    masks_dir='Data/masks',
    output_dir='Data/images2'
)


✅ 288 images copied.
⏭️ 923 images skipped (no matching mask).


In [2]:
import os
import glob

# --- Dossier où sont stockées les images ---
image_dir = "../Data/masks"

# --- Parcourir tous les fichiers .JPG ---
for file in glob.glob(os.path.join(image_dir, "*.JPG")):
    # Nouveau nom avec .jpg
    new_file = file.replace(".JPG", ".jpg")
    
    # Renommer
    os.rename(file, new_file)
    print(f"Renommé: {file} -> {new_file}")


Renommé: ../Data/masks/L3_1_perp_45_mask_Latuale_0709.JPG -> ../Data/masks/L3_1_perp_45_mask_Latuale_0709.jpg
Renommé: ../Data/masks/L3_1_perp_57_mask_Lataule_0709.JPG -> ../Data/masks/L3_1_perp_57_mask_Lataule_0709.jpg
